El deportista mejor pagado del mundo

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime

In [2]:
df = pd.read_csv('forbes-dataset-2020.csv')
df.head()

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0


In [3]:
df1 = df.copy()
df1.drop('S.NO',axis=1,inplace=True)
df1.info()
# Convert string to datetime64
df1['Year'] = df1['Year'].apply(pd.to_datetime,format='%Y')
df1["Year"]
#Set Date column as the index column.
df1['year'] = pd.DatetimeIndex(df1['Year']).year  
df1.set_index('year', inplace=True)
df1.drop('Year',axis=1,inplace=True)
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  301 non-null    object 
 1   Nationality           301 non-null    object 
 2   Current Rank          301 non-null    int64  
 3   Previous Year Rank    277 non-null    object 
 4   Sport                 301 non-null    object 
 5   Year                  301 non-null    int64  
 6   earnings ($ million)  301 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 16.6+ KB


,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
1990,Mike Tyson,USA,1,NaN,boxing,28.6
1990,Buster Douglas,USA,2,NaN,boxing,26.0
1990,Sugar Ray Leonard,USA,3,NaN,boxing,13.0
1990,Ayrton Senna,Brazil,4,NaN,auto racing,10.0
1990,Alain Prost,France,5,NaN,auto racing,9.0
...,...,...,...,...,...,...
2020,Stephen Curry,USA,6,9,Basketball,74.4
2020,Kevin Durant,USA,7,10,Basketball,63.9
2020,Tiger Woods,USA,8,11,Golf,62.3


1. deportistas cuyo ranking ha subido al menos dos lugares entre 2010 y 2020

In [4]:
#generamos copia de datos 
df_rank = df1.copy()
#seleccionamos datos desde 2010
df_rank = df_rank[df_rank.index > 2009]
#contamos cuantas veces está en el ranking
sub_df_rank = df_rank["Name"].value_counts().to_frame()
#solo tomaremos los nombres de los 
nombres = list(sub_df_rank.loc[sub_df_rank["Name"]>1].index)

In [5]:
# creamos funcion para obtener deportistas que subieron al menos dos lugares 
def subir_ranking(nombres):
  resultado = []
  for nombre in nombres:
    #sacamos diferencia de max y min
    max_rank = df_rank[df_rank["Name"] == nombre]["Current Rank"].max() 
    min_rank = df_rank[df_rank["Name"] == nombre]["Current Rank"].min()
    dif = max_rank - min_rank
    #si pudo subir 2 o más se guarda valor en lista
    if dif > 1:
      resultado.append(nombre)
  return(resultado)

In [6]:
# Aplicamos la funcion anteriro para obtener a dichos deportistas
subir_ranking(nombres)
df1.loc[df1["Name"].isin(nombres)].head()

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
1997,Tiger Woods,USA,6,26,Golf,26.1
1998,Tiger Woods,USA,4,NaN,Golf,26.8
1999,Tiger Woods,USA,2,4,Golf,47.0
2000,Tiger Woods,USA,2,2,Golf,53.0
2002,Tiger Woods,USA,1,2,Golf,69.0


2. Atleta con el menor número de apariciones y mayores ganancias.

In [7]:
#contamos apariciones
df2 = df1["Name"].value_counts().to_frame()
#seleccionamos a quienes aparezce una vez
sub_atletas = df2[df2["Name"]<2].index
#seleccionamos filas de dichos atletas
sub_2 = df1.loc[df1["Name"].isin(sub_atletas)]
# ordenamos por orden de ganancia
sub_2 = sub_2.sort_values(by="earnings ($ million)", ascending = False)
#veamos a los primeros 5
sub_2[:5]

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
2018,Conor McGregor,Ireland,4,24,MMA,99.0
2019,Canelo Alvarez,Mexico,4,15,Boxing,94.0
2019,Russell Wilson,USA,6,??,American Football,89.5
2019,Aaron Rogers,USA,7,??,American Football,89.3
2020,Kirk Cousins,USA,9,>100,American Football,60.5


3. Deporte y país con mayor número de atletas no rankeados que entraron en la lista de atletas mejor pagados.

In [8]:
# pedimos subset donde el rank fuera desconocido o mayor a 100
noranked = df1.loc[(df1["Previous Year Rank"] == "??") | (df1["Previous Year Rank"] == ">100")]
noranked[["Nationality","Sport"]]


,Nationality,Sport
year,,
2014,USA,American Football
2018,USA,Boxing
2018,USA,American Football
2018,USA,American Football
2019,USA,American Football
2019,USA,American Football
2020,USA,American Football
2020,USA,American Football


4. País con mayor número de deportes con atletas en el dataset.


In [9]:
#para homogenizar todos los nombres pasaremos todo a minusculas
df1["Sport"] = df1["Sport"].str.lower()

In [10]:
#agrupamos por nacionalidad
Paises_mayor = df1.groupby("Nationality").agg(
    {"Sport":"nunique","Name":"nunique"}).sort_values(
        by=["Name","Sport"], ascending = False)
#veamos el primer pais
Paises_mayor


,Sport,Name
Nationality,,
USA,13,56
Canada,4,4
Brazil,2,3
UK,2,3
Germany,3,1
Argentina,1,1
Australia,1,1
Austria,1,1
Dominican,1,1


In [11]:
#indice a columna
Paises_mayor = Paises_mayor.rename_axis('Nationality').reset_index()

Paises_mayor.to_csv('paises_mayor.csv', index = False)

5. ¿Cuántos atletas por deporte tiene cada país?


In [12]:
#agrupamos por nacionalidad y deportes, contamos atletas , nunique para solo contar unicos
atletas_deporte_pais =  df1.groupby(["Nationality", "Sport"]).agg({"Name":"nunique"})
atletas_deporte_pais

Name
Nationality      Sport                             
Argentina        soccer                           1
Australia        golf                             1
Austria          auto racing                      1
Brazil           auto racing                      1
                 soccer                           2
Canada           auto racing                      1
                 boxing                           1
                 hockey                           1
                 ice hockey                       1
Dominican        baseball                         1
Filipino         boxing                           1
Finland          f1 racing                        1
France           auto racing                      1
Germany          auto racing                      1
                 f1 motorsports                   1
                 f1 racing                        1
Ireland          mma                              1
Italy            motorcycle gp                    1
Mexico           boxing                           1
Northern Ireland golf                             1
Philippines      boxing                           1
Portugal         soccer                           1
Russia           ice hockey                       1
Serbia           tennis                           1
Spain            tennis                           1
Switzerland      tennis                           1
UK               auto racing                      2
                 soccer                           1
USA              american football               15
                 american football / baseball     1
                 auto racing                      1
                 auto racing (nascar)             2
                 baseball                         3
                 basketball                      12
                 boxing                          11
                 cycling                          1
                 golf                             6
                 nascar                           2
                 nba                              1
                 nfl                              2
                 tennis                           3

In [24]:
#atletas_deporte_pais  = atletas_deporte_pais.rename_axis('Nationality').reset_index()

#Paises_mayor.to_csv('atletas_deporte_pais.csv', index = False)

6. Ganancia mínima y máxima (dentro del dataset) por deporte y por país.


In [14]:
ganancia_minmax = df1.groupby(["Nationality", "Sport"]).agg({"earnings ($ million)":[min, max]})

In [15]:
ganancia_minmax

earnings ($ million)       
                                                               min    max
Nationality      Sport                                                   
Argentina        soccer                                       32.3  127.0
Australia        golf                                          8.5    8.5
Austria          auto racing                                  13.5   13.5
Brazil           auto racing                                  10.0   22.0
                 soccer                                       31.0  105.0
Canada           auto racing                                  20.0   23.0
                 boxing                                       10.2   10.2
                 hockey                                       17.9   17.9
                 ice hockey                                   13.5   14.5
Dominican        baseball                                     35.0   35.0
Filipino         boxing                                       62.0   62.0
Finland          f1 racing                                    40.0   45.0
France           auto racing                                   9.0   16.0
Germany          auto racing                                  15.0   59.0
                 f1 motorsports                               33.0   75.0
                 f1 racing                                    34.0   80.0
Ireland          mma                                          99.0   99.0
Italy            motorcycle gp                                28.0   35.0
Mexico           boxing                                       94.0   94.0
Northern Ireland golf                                         50.0   50.0
Philippines      boxing                                       40.0  160.0
Portugal         soccer                                       38.0  109.0
Russia           ice hockey                                   29.8   29.8
Serbia           tennis                                       55.8   55.8
Spain            tennis                                       44.5   44.5
Switzerland      tennis                                       35.0  106.3
UK               auto racing                                   9.0   46.0
                 soccer                                       28.0   50.0
USA              american football                            29.0   89.5
                 american football / baseball                 22.5   22.5
                 auto racing                                  26.5   26.5
                 auto racing (nascar)                         20.0   24.5
                 baseball                                     12.7   29.0
                 basketball                                    8.1   89.0
                 boxing                                        8.1  300.0
                 cycling                                      28.0   28.0
                 golf                                          8.1  115.0
                 nascar                                       19.1   34.0
                 nba                                          15.2   15.2
                 nfl                                           9.5   16.5
                 tennis                                        8.5   28.2

7. Atleta con mayores ganancias por deporte por década


In [16]:
#agrupamos por deporte, contamos atletas , nunique para solo contar unicos
decada1 = df1[df1.index < 2000]
decada3 = df1[df1.index >= 2010]
decada2 = df1[(df1.index > 1999) & (df1.index < 2010)]
#df1.groupby(["Sport"]).agg({"Name":"nunique"})

In [17]:
#agrupamos por deporte, por nombre y sumamos sus ganancias 
Mejores_decada1=decada1.groupby(["Sport","Name"]).agg({"earnings ($ million)":sum}).sort_values(
        by=["earnings ($ million)"], ascending = False)
Mejores_decada1

earnings ($ million)
Sport                        Name                                         
basketball                   Michael Jordan                          409.8
boxing                       Mike Tyson                              235.1
                             Evander Holyfield                       213.9
f1 motorsports               Michael Schumacher                      106.0
boxing                       Oscar De La Hoya                        100.0
golf                         Tiger Woods                              99.9
                             Arnold Palmer                            75.3
basketball                   Shaquille O'Neal                         71.7
auto racing                  Michael Schumacher                       64.0
                             Ayrton Senna                             63.5
basketball                   Grant Hill                               61.6
tennis                       Andre Agassi                             53.6
boxing                       George Foreman                           48.3
basketball                   Shaq O'Neal                              47.7
boxing                       Riddick Bowe                             47.2
                             Lennox Lewis                             44.0
nascar                       Dale Earnhardt                           43.2
auto racing                  Alain Prost                              36.0
                             Nigel Mansell                            34.8
golf                         Sugar Ray Leonard                        32.8
basketball                   Patrick Ewing                            31.2
ice hockey                   Sergei Federov                           29.8
                             Wayne Gretzky                            28.0
auto racing                  Dale Earnhardt                           26.5
boxing                       Buster Douglas                           26.0
golf                         Jack Nicklaus                            23.4
american football / baseball Deion Sanders                            22.5
tennis                       Jim Courier                              21.6
nfl                          Joe Montana                              21.0
hockey                       Joe Sakic                                17.9
baseball                     Gary Sheffield                           17.2
nfl                          Emmit Smith                              16.5
nba                          Shaquille O'Neal                         15.2
auto racing                  Gerhard Berger                           13.5
boxing                       Sugar Ray Leonard                        13.0
basketball                   Dennis Rodman                            12.9
baseball                     Cecil Fielder                            12.7
boxing                       Michael Moorer                           12.1
                             Donovan "Razor" Ruddock                  10.2
golf                         Greg Norman                               8.5
tennis                       Monica Seles                              8.5

In [18]:
Mejores_decada2=decada2.groupby(["Sport","Name"]).agg({"earnings ($ million)":sum}).sort_values(
        by=["earnings ($ million)"], ascending = False)
Mejores_decada2

,,earnings ($ million)
Sport,Name,
golf,Tiger Woods,782.3
basketball,Michael Jordan,329.0
f1 racing,Michael Schumacher,234.0
basketball,Kobe Bryant,224.9
soccer,David Beckham,185.5
golf,Phil Mickelson,174.0
basketball,Shaquille O'Neal,151.3
f1 motorsports,Michael Schumacher,142.0
boxing,Oscar De La Hoya,134.0


In [19]:
Mejores_decada3=decada3.groupby(["Sport","Name"]).agg({"earnings ($ million)":sum}).sort_values(
        by=["earnings ($ million)"], ascending = False)
Mejores_decada3

,,earnings ($ million)
Sport,Name,
boxing,Floyd Mayweather,840.0
soccer,Cristiano Ronaldo,787.1
basketball,LeBron James,766.8
tennis,Roger Federer,746.1
soccer,Lionel Messi,715.5
golf,Tiger Woods,491.6
basketball,Kobe Bryant,376.2
golf,Phil Mickelson,345.9
basketball,Kevin Durant,300.3


8. Ganancia total por cada deporte por cada año.


In [20]:
# agrupar por deporte sumar earnings
df1.groupby(["Sport"]).agg({"earnings ($ million)":sum})

,earnings ($ million)
Sport,
american football,901.2
american football / baseball,22.5
auto racing,386.3
auto racing (nascar),44.5
baseball,146.1
basketball,3418.1
boxing,2298.8
cycling,28.0
f1 motorsports,248.0


In [22]:
df1_nuevo =df1.rename_axis('year').reset_index()


In [23]:
df1_nuevo.to_csv('datos_forbes_limpios.csv', index = False)
df1_nuevo

,year,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
0,1990,Mike Tyson,USA,1,NaN,boxing,28.6
1,1990,Buster Douglas,USA,2,NaN,boxing,26.0
2,1990,Sugar Ray Leonard,USA,3,NaN,boxing,13.0
3,1990,Ayrton Senna,Brazil,4,NaN,auto racing,10.0
4,1990,Alain Prost,France,5,NaN,auto racing,9.0
...,...,...,...,...,...,...,...
296,2020,Stephen Curry,USA,6,9,basketball,74.4
297,2020,Kevin Durant,USA,7,10,basketball,63.9
298,2020,Tiger Woods,USA,8,11,golf,62.3
299,2020,Kirk Cousins,USA,9,>100,american football,60.5
